# Get the data   
Note: automate as much as possible so you can easily get fresh data.

- [x] List the data you need and how much you need.  

> * FMI API weather forecast data from fmi_weather app.
> * FMI historical data downloaded with a csv from website
> * Helen historical data downloaded with a csv from website
> * Calculation of theoretical solar irradiance

- [x] Find and document where you can get that data. 
>in 1.
- [x] Check how much space it will take.
>Negligible
- [x] Check legal obligations, and get the authorization if necessary.
>* fmi_weather license
>* cite the GHI article
>* helen data and weather observation data are freely available
- [x] Get access authorizations.
>Wasn't necessary
- [x] Create a workspace (with enough storage space).
- [x] Get the data.
- [x] Convert the data to a format you can easily manipulate (without changing the data itself).
- [x] Ensure sensitive information is deleted or protected (e.g., anonymized).
- [x] Check the size and type of data (time series, sample, geographical, etc.).
>Timeseries converted into UTC
- [ ] Sample a test set, put it aside, and never look at it (no data snooping!).
>Do that
